In [ ]:
import json
import numpy as np
import pandas
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
import os
import psycopg2
import psycopg2.extensions

from py2neo import Graph
from twitter import Twitter, OAuth

from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import random
import csv
from string import strip
from math import floor, ceil

# for sending offers via gmail
import base64
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib

# Natural language query parser
import parse_query
    
import pymongo
client = pymongo.MongoClient()
db = client.customer_database

with open('config.json') as config_file:
    config = json.load(config_file)

graph = Graph(config["neo4j"])

if 'http_proxy' in os.environ:
    if 'https_proxy' not in os.environ:
        os.environ["https_proxy"] = os.environ["http_proxy"]

twitter = Twitter(auth=OAuth(config['token'], config['token_secret'], config['consumer_key'], config['consumer_secret']))

field_names =  ['user_screen_name', 'user_name', 'id_str', 'created_at', 
                'sentiment', 'categories', 'text', 'klout_score', 'segment_id',
               'got_reply', 'followers_count', 'is_agent_reply']

segment_dict = {0:'-', 1:'Mass Market', 2:'Young Professional',
                3:'Mass Afluent', 4:'Affluent', 5:'High Net Worth'}

#suboffers = {} # offers dictionary
#with open('offers_suboffers.csv', 'rb') as csvfile:
#    file_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
#    for row in file_reader:
#        if len(row) >= 3:          # check if we have at least 3 field
#            key = strip(row[0])
#            value = strip(row[1])
#            imageId = strip(row[2])
#            text = strip(row[3]) if len(row) >= 4 else ""
#            if key in suboffers:
#                suboffers[key].append({"name": value, "image": imageId, "text":text})
#            else:
#                suboffers[key] = [{"name": value, "image": imageId, "text":text}]

def get_priority_key(tw_l):
    for tw in tw_l:
        if not tw['is_agent_reply']:
            p = (tw['segment_id']/2 - tw['sentiment'] - tw['got_reply']*5 +
                tw['followers_count']/1000 + int('fraud' in tw['categories']))
            return -p

In [ ]:
from flask import Flask, render_template, jsonify, request, send_file
from flask_sockets import Sockets
from json import dumps
import gevent
import time
import StringIO

app = Flask(__name__)
sockets = Sockets(app)

@app.route('/voc')
def voc():
    try:
        return render_template("voc.html")
    except Exception, e:
        return str(e)

@app.route('/720page/<name>')
def foo_720page(name):
    try:
        return render_template("720page.html", name=name)
    except Exception, e:
        return str(e)

@app.route('/720view/<name>')
def foo_720view(name):
    try:
        """
        customer = db.customers.find_one(
            { 'name' :  name},
            {'_id':  0,
             'photo':1,
             'name':1,
             'phone':1,
             'status':1}
        )
        """
        if name.lower()[:8]=='kathleen':
            customer = {
                'name': 'Kathleen Fanning',
                'photo': '/static/images/photo_female.png',
                'phone': '212-2222-245',
                'status': 'Married'
            }
        else:
            customer = {
                'name': 'John Smith',
                'photo': '/static/images/photo_male.png',
                'phone': '212-2222-245',
                'status': 'Married'
            }
        return render_template("720view.html", customer=customer)
    except Exception, e:
        return str(e)

    
"""
@sockets.route('/twitter_stream')
def twitter_stream(ws):
    j = 0
    
    conn = psycopg2.connect(database="twitter", user="postgres")
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()
    curs.execute("LISTEN new_tweet;")
    
    while not ws.closed:
        conn.poll()
        t0 = time.time()
        if conn.notifies:
            notify = conn.notifies.pop(0)
            
            ws.send("{notification:\'new tweet\'}")
            j+=1
        if time.time() - t0 > 3600:
            print("1hr timeout")
            break
        else:
            gevent.sleep()
    conn.close()
"""


@app.route('/reply', methods=['POST'])
def profile():
    data = eval(request.form['py_data'])
    status = request.form['status']
    status_id = request.form['status_id']
    
    sentiment = data['sentiment']
    user_name = data['user_name']
    
    tweet = twitter.statuses.update(status=status, in_reply_to_status_id=status_id)
    
    conn = psycopg2.connect(database=config["twitter_db"]["database"], user=config["twitter_db"]["user"])
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()
    
    curs.execute("""update customers
                set got_reply=1
                where user_screen_name=%s""",(tweet['in_reply_to_screen_name'],))
    
    curs.execute("""insert into stream (id_str, text, created_at, in_reply_to_status_id,
                in_reply_to_screen_name, user_screen_name, user_name, sentiment, is_agent_reply)
                values (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
                 (tweet['id_str'],
                  tweet['text'],
                  tweet['created_at'],
                  tweet['in_reply_to_status_id'],
                  tweet['in_reply_to_screen_name'],
                  tweet['in_reply_to_screen_name'],
                  user_name,
                  sentiment,
                  1,))
    
    conn.close()

    return 'good'

@app.route('/send_offer', methods=['POST'])
def send_offer():
    recipient_name = request.form['recipient-name']
    message_text = request.form['message-text']
    offer_image_id = request.form['offer-image-id']

    gmail_user = config["gmail_user"]
    gmail_pass = config["gmail_pass"]
    gmail_recipient = config["gmail_recipient"]
    TO = gmail_recipient if type(gmail_recipient) is list else [gmail_recipient]

    msg = MIMEMultipart('alternative')
    msg['Subject'] = "Special offer from CapBank"
    msg['From'] = "VIP clients support team"
    msg['To'] = recipient_name
    
    fp = open("static/images/email/email_tmp"+offer_image_id+".png", 'rb')
    img_b64 = base64.b64encode(fp.read()).decode('ascii')
    fp.close()

    html = """<html>
      <head></head>
      <body>
        <img src="data:image/png;base64,{0}" /><br />
        <pre>{1}</pre>
      </body>
    </html>""".format(img_b64, message_text)
    
    # Record the MIME types of both parts - text/plain and text/html.
    #part1 = MIMEText(text, 'plain')
    part2 = MIMEText(html, 'html')    
    
    # Attach parts into message container.
    # According to RFC 2046, the last part of a multipart message, in this case
    # the HTML message, is best and preferred.
    #msg.attach(part1)
    msg.attach(part2)

    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.ehlo()
        server.starttls()
        server.login(gmail_user, gmail_pass)
        server.sendmail(gmail_user, gmail_recipient, msg.as_string())
        server.close()
        res = '<h4 class="bg-success">E-mail sent successfully</h4>'
    except:
        res = '<h4 class="bg-danger">E-mail sending failed</h4>'

    return res

@app.route('/delete_tweets', methods=['GET'])
def delete_tweets():
    try:
        conn = psycopg2.connect(database="twitter", user="postgres")
        conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
        curs = conn.cursor()

        curs.execute("DELETE FROM stream WHERE user_screen_name = 'capgbanktest';")
        conn.close()
        res = 'Deleted all tweets for `capgbanktest`'
    except:
        res = 'Failed to delete tweets'

    return res

@app.route('/data', methods=['GET'])
def provide_data():
    conn = psycopg2.connect(database="twitter", user="postgres")
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()

    curs.execute("""select stream.user_screen_name, stream.user_name, stream.id_str,
                stream.created_at, stream.sentiment, stream.categories, stream.text,
                customers.klout_score, customers.segment, customers.got_reply, 
                customers.followers_count, stream.is_agent_reply
                from stream
                INNER JOIN customers
                ON customers.user_screen_name=stream.user_screen_name
                where stream.user_screen_name in 
                (
                    select tmp.user_screen_name
                    from (
                        select user_screen_name as user_screen_name, max(id_str) as max_id_str
                        from stream
                        group by user_screen_name
                    ) tmp
                    left join (
                        select user_screen_name, got_reply
                        from customers
                    ) tmp2
                    on tmp.user_screen_name=tmp2.user_screen_name
                    order by tmp2.got_reply asc, tmp.max_id_str desc limit 10
                )
                order by customers.got_reply desc, stream.id_str desc
                ;""")
    rec = curs.fetchall()
    conn.close()
    
    user_blocks = {v[0]:[] for v in rec}
    for row in rec:
        if len(user_blocks[row[0]]) < 10:
            user_blocks[row[0]] += [{k:v for k,v in zip(field_names, row)}]
            user_blocks[row[0]][-1]['id_str'] = str(user_blocks[row[0]][-1]['id_str'])
            user_blocks[row[0]][-1]['segment'] = segment_dict[user_blocks[row[0]][-1]['segment_id']]
    
    user_blocks_v = user_blocks.values()
    user_blocks_v_s = sorted(user_blocks_v, key=get_priority_key)
    
    return jsonify({'data':user_blocks_v_s})

@app.route('/graph')
def get_graph():
    results = graph.cypher.execute(
        "match (p)-[r]->(p1) "
        "where ((r.favorited>0 and r.retweeted>0) or (r.favorited>0 and r.replied>0) or (r.retweeted>0 and r.replied>0) "
        "or (r.favorited>1) or (r.retweeted>1) or (r.replied>1)) and (not p=p1) "
        "return p.screen_name, sum(r.favorited)+sum(r.retweeted)+sum(r.replied) as cardinality, p1.screen_name "
        "limit 500;")
    nodes = []
    rels = []

    persons = {}
    for person_b, cardinality, person_a in results:
        if person_a in persons:
            persons[person_a] += 1
        else:
            persons[person_a] = 1

        if person_b not in persons:
            persons[person_b] = 1

    for i, v in persons.iteritems():
        if i == u'gunjan_amit':
            person = {
                "id": persons.keys().index(i),
                #"caption": i, 
                "caption": "User", 
                "role": "center"}
        else:
            person = {
                "id": persons.keys().index(i),
                "caption": i, 
                "role": "customer" if np.random.rand()>0.9 else "user",
                }
        nodes.append(person)

    for person_b, cardinality, person_a in results:
        person = {"caption": person_a, "role": "customer"}
        source = persons.keys().index(person_a)
        target = persons.keys().index(person_b)
        rels.append({"source": source, 
            "target": target, 
            "weight": cardinality,
            "caption": "Value: {0}".format(cardinality)
            })

    return jsonify({"comment": "Tweeter users graph", "nodes": nodes, "edges": rels})

@app.route('/download/customers')
def index():
    strIO = StringIO.StringIO()
    strIO.write('Customer Name, Status, Churn Rate\n')
#    for i in range(10):
#        strIO.write(str(np.floor(10000*np.random.rand()))+',Credit Card\n')
    
    cursor = db.customers.find().limit(103)
    
    for  c in cursor:
        strIO.write(str(c['name'])+','+str(c['status'])+','+str(round(c['churn_rate']*100))+'%\n')

    strIO.seek(0)
    return send_file(strIO,
                     attachment_filename="Customers.csv",
                     as_attachment=True)

@app.route('/map/get_customers/<state>')
def map_get_customers(state):
    try:
        #cursor = db.customers.find({ 'state' : state })
        cursor = db.customers.find(
            { 'state' : state },
            {'_id':  0,'customerId':1,'name':1,'churn_rate':1,'status':1}
        )

        status_weights = {'platinum':10,'gold':9,'silver':8,'bronze':7}

        customers = []
        for c in cursor:
            c.update({'importance':status_weights[c['status'].lower()]*c['churn_rate']})
            customers.append(c)

        customers = sorted(customers, key=lambda x:-x['importance'])[:100]

        customers = [[c['importance'],
                      c['customerId'],
                      c['name'],
                      str(int(c['churn_rate']*100))+' %',
                      c['status'],] 
                     for c in customers]
        # Set selected user
        customers[0][1] = 1
        customers[0][2] = 'Kathleen Fanning'
        customers[0][3] = '71 %'
        return jsonify({'data':customers})
    
    except Exception, e:
        return str(e)

@app.route('/map/get_states_data')
def map_states_data():
    try:
        with open('static/js/us-states.json') as f:
            us_states_obj = json.load(f)

        states_data = db.customers.aggregate([
            {"$group": {"_id": "$state", 
                        'averageChurn': { '$avg': "$churn_rate" },
                        "count": {"$sum": 1}}}])
        states_data = {row['_id']:row for row in states_data}

        states_data_objs = []

        acc = 0
        for i in range(len(us_states_obj['features'])):
            state_name = us_states_obj['features'][i][u'properties']['name']
            if state_name in states_data:
                us_states_obj['features'][i]['properties'].update(states_data[state_name])
                states_data_objs.append(us_states_obj['features'][i])

        us_states_obj['features'] = states_data_objs

        return jsonify(us_states_obj)
    except Exception, e:
        return str(e)

@app.route('/map')
def mappage():
    try:
        return render_template("map.html")
    except Exception, e:
        return str(e)

@app.route('/cluster')
def clusterpage():
    try:
        return render_template("cluster.html")
    except Exception, e:
        return str(e)

@app.route('/nbo/<int:userid>')
def nbopage(userid):
    try:
        cursor = db.customers.find({ 'customerId' : userid })
        if cursor.count() < 1:
            return render_template("nbo_no_such_user.html")
        mongo_record = cursor.next()
    except Exception, e:
        return str(e)

    customer = {}
    # General fields
    data_list = ['name', 'status', 'state', 'maritalStatus', 'creditScoreFICO', 'email', 
                 'numberOfChildren', "predictedLifeEvent"]
    for l in data_list:
        customer[l] = mongo_record[l] if l in mongo_record else 'N/A' 

    # Fields with money
    data_list = ['investableAssetEstimate', 'zillowHomeValueEstimation', 'homeEquityEstimate']
    for l in data_list:
        customer[l] = "$ {0:,d}".format(int(mongo_record[l])) if l in mongo_record else 'N/A'
    
    
    # "Profitability" time series
    timeseries = mongo_record['monthlyProfitabilities']

    month_list = []
    this_year_values = []
    last_year_values = []

    if len(timeseries) > 1:
        timeseries = sorted(timeseries, key=lambda r: r['date'], reverse = True) # sort by year+month
        timeseries = timeseries[:20] # keep no more than 20 last months

        # make our data look like they are fresh
        data_date = date(int(timeseries[0]['date'][:4]), int(timeseries[0]['date'][5:]), 1)
        our_date = date.today()-relativedelta(months=1)

        for i in range(0, 4):
            month_list.append(our_date.strftime("%b"))

            look_for_date = "{0}-{1:02d}".format(data_date.year, data_date.month)
            profitability = next( (r['profitability'] 
                                   for r in timeseries 
                                   if r['date']==look_for_date)
                                 , None)
            this_year_values.append(profitability)

            look_for_date = "{0}-{1:02d}".format(data_date.year-1, data_date.month)
            profitability = next( (r['profitability'] 
                                   for r in timeseries 
                                   if r['date']==look_for_date)
                                 , None)
            last_year_values.append(profitability)

            our_date -= relativedelta(months=1)
            data_date -= relativedelta(months=1)

        month_list.reverse()
        this_year_values.reverse()
        last_year_values.reverse()
    
    customer['profitability_months'] = month_list
    customer['profitability_this_year'] = this_year_values
    customer['profitability_last_year'] = last_year_values

    # "Products" time series
    timeseries = mongo_record['products']
    month_list = []
    investment_values = []
    savings_values = []
    checking_values = []

    if len(timeseries) > 1: 
        timeseries = sorted(timeseries, key=lambda r: r['yearAndMonth'], reverse = True)
        timeseries = timeseries[:60] # keep no more than 60 last values (up to 20 months)

        # make our data look like they are fresh
        data_date = date(int(timeseries[0]['yearAndMonth'][:4]), # year
                         int(timeseries[0]['yearAndMonth'][5:]), # month
                         1)                                      # day
        our_date = date.today()-relativedelta(months=1)

        for i in range(0, 4):
            month_list.append(our_date.strftime("%b"))

            look_for_date = "{0}-{1:02d}".format(data_date.year, data_date.month)
            investment = next( (r['monthlyAverage'] 
                                for r in timeseries 
                                if r['yearAndMonth']==look_for_date and r['product']=="Investment")
                              , None)
            investment_values.append(investment)
            savings = next( (r['monthlyAverage'] 
                                for r in timeseries 
                                if r['yearAndMonth']==look_for_date and r['product']=="Savings")
                              , None)
            savings_values.append(savings)
            checking = next( (r['monthlyAverage'] 
                                for r in timeseries 
                                if r['yearAndMonth']==look_for_date and r['product']=="Checking")
                              , None)
            checking_values.append(checking)

            our_date -= relativedelta(months=1)
            data_date -= relativedelta(months=1)

        month_list.reverse()
        investment_values.reverse()
        savings_values.reverse()
        checking_values.reverse()

    customer['products_months'] = month_list
    customer['investment_values'] = investment_values
    customer['savings_values'] = savings_values
    customer['checking_values'] = checking_values

    # "Surveys" time series
    timeseries = mongo_record['surveys']
    surveys_data = []
    last4years = []
    
    if len(timeseries) > 0: 
        years_show = 4
        timeseries = sorted(timeseries, key=lambda r: r['surveyDate'], reverse = True)
        timeseries = timeseries[:years_show] # keep no more than 4 last values

        # make our data look like they are for last 4 years
        date_to = date.today()
        for r in timeseries:
            date_to = date_to-relativedelta(years=1)
            surveys_data.append(r['surveyResult'])
            last4years.append(date_to.year)
        # fill empty values with None
        for i in range(len(timeseries), years_show):
            date_to = date_to-relativedelta(years=1)
            surveys_data.append(None)
            last4years.append(date_to.year)

    surveys_data.reverse()
    last4years.reverse()
    customer['surveys_data'] = surveys_data
    customer['last4years'] = last4years

    nbo_offers = mongo_record['nextBestOffers']
    if len(nbo_offers) > 0: 
        nbo_offers = sorted(nbo_offers, key=lambda r: r['nboProbability'], reverse = True)
        i = 1
        for r in nbo_offers:
            r['nboProbability'] = "{0:.1f}%".format(r['nboProbability']*100.0)
            r['index'] = i
            i += 1
    customer['nbo_offers'] = nbo_offers
    
#    customer['suboffers'] = suboffers

    # List of last 4 months
    customer['last4months'] = [(date.today()-relativedelta(months=a)).strftime('%b') for a in range(4,0,-1)]

    try:
        return render_template("nbo.html", customer=customer)
    except Exception, e:
        return str(e)

@app.route('/loopfeedback')
def loopfeedbackpage():
    try:
        return render_template("loopfeedback.html")
    except Exception, e:
        return str(e)

@app.route('/loyalty')
def loyaltypage():
    try:
        return render_template("loyalty.html")
    except Exception, e:
        return str(e)

@app.route('/wm', methods=['GET'])
def wmpage():
    try:
        name = ''
        status = 'ok'
        if 'name' in request.args:
            name = request.args['name']
            if name.lower() == 'deborah winshel':
                name = 'Deborah Winshel'
            else:
                name = str(name)
                status = 'Not found'
        else:
            status = 'Search'
        
        return render_template("wm.html", name=name, status=status)
    except Exception, e:
        return str(e)

@app.route('/pci')
def pcipage():
    try:
        return render_template("pci.html")
    except Exception, e:
        return str(e)

@app.route('/send_pci_query', methods=['POST'])
def send_pci_query():
    query_text = request.form['query']

    def err_msg(text = 'Cannot understand you query, please try to paraphrase it'):
        return jsonify({'error': 'yes', 'error_text': text,  'query': query_text})

    def one_hot_dataframe(data, cols, replace=False):
        vec = DictVectorizer()
        mkdict = lambda row: dict((col, row[col]) for col in cols)
        vecData = pandas.DataFrame(vec.fit_transform(data[cols].apply(mkdict, axis=1)).toarray())
        vecData.columns = vec.get_feature_names()
        vecData.index = data.index
        if replace is True:
            data = data.drop(cols, axis=1)
            data = data.join(vecData)
        return (data, vecData, vec)


    query = parse_query.parse_it(query_text)

    # check if it is predictive query
    if query and 'parameters' in query and 'mode' in query['parameters'] and query['parameters']['mode'] == 'model':
        #search for target variable
        target = None
        target_variables = ['debt_funds', 'credit_card']
        new_filter = []
        for sel in query['sql_filter']:
            for var in target_variables:
                if var in sel:
                    target = var
                    break 
            else: # do not keep target variable in filter list (for-else python structure)
                new_filter.append(sel) 

        if target == None: 
            return err_msg('Cannot understand you query, please try to paraphrase it') # couldn't find target 
        
        parameters = set(["age" , "job" , "marital" , "education" , "debt_funds" , "credit_card" , 
                          "income" , "state" , "gender"])
        parameters = list(parameters - set([target]))
        
        sql_query = "SELECT " + target + ", " + ", ".join(parameters) + " FROM pci_customers"
        if new_filter: sql_query += " WHERE " + " AND ".join(new_filter)

        conn = psycopg2.connect(database="pci", user="postgres")

        try:
            full_df = pandas.read_sql(sql_query, conn)
        except Exception, e:
            return err_msg(str(e))

        conn.close()
        
        y_name = full_df.keys()[0]
        x = full_df.drop(y_name, axis=1)
        y = full_df[y_name].copy()
        
        y.replace(to_replace=['no', 'yes'], value=[0, 1] , inplace = True)
        x.replace(to_replace=['no', 'yes'], value=[0, 1] , inplace = True)
        x.replace(to_replace=['F', 'M'], value=[0, 1] , inplace = True)
        
        x2, _, _ = one_hot_dataframe(x, ['state', 'marital', 'job', 'education'], replace=True)
        
        dtc = DecisionTreeClassifier(random_state=241, max_leaf_nodes=40, criterion='gini')
        dtc.fit(x2, y)
        feature_w = dict() # Feature weights
        for i in range(len(x2.columns)):
            if '=' in x2.columns[i]:
                name = x2.columns[i].split('=')[0] # One-hot feature names are like 'job=student'
            else:
                name = x2.columns[i]
            name = name.replace('_', ' ').capitalize()
            if name in feature_w:
                feature_w[name] += dtc.feature_importances_[i]
            else:
                feature_w[name] = dtc.feature_importances_[i]
        
        # Convert to descending list
        #feature_w = sorted(feature_w.items(), key = lambda x: -x[1])
        
        # Convert to list (when unordered it looks better on spider-web type chart)
        feature_w = feature_w.items()
        
        # fill with default data
        charts = {'age': 0,
                 'gender': 0,
                 'state': 0,
                 'credit_card': 0,
                 'debt_funds': 0,
                 'job': 0,
                 'income': 0,
                 'education': 0,
                 'marital': 0}



        # Predictors is list of tuples (<parameter>, <Parameter name>)
        predictors = [('age', 'Age'), ('income', 'Income'), ('job', 'Job type'), ('education', 'Education level'),
                     ('marital', 'Marital status'), ('gender', 'Gender')]
        # Collect data
        for predictor, pr_name in predictors:
            if predictor == 'age':
                series = full_df.groupby([ y_name, np.floor(full_df[predictor]/10)*10 ]).count()['age']
            elif predictor == 'income':
                series = full_df.groupby([ y_name, np.floor(full_df[predictor]/1000)*1000 ]).count()['age']
            else:
                series = full_df.groupby([y_name, predictor]).count()['age']
            
            res_series = []
            for i in series.index.levels[0]:
                series_data = []
                for k in series.index.levels[1]:
                    if (i, k) in series:
                        # append result as percentage
                        series_data.append( floor(1000.0*series[(i, k)]/x2.shape[0])/10 )
                    else:
                        series_data.append( None )
                series_name = "Positive" if i == "yes" else "Negative"
                visibility = True if i == "yes" else False
                res_series.append({"name": series_name, "data": series_data, "visible": visibility})
            res_series.reverse()


            params = {'name': pr_name}
            if predictor == 'gender':
                params['categories'] = [w.replace('F', 'Female').replace('M', 'Male') for w in series.index.levels[1]]
            else:
                params['categories'] = list(series.index.levels[1])
            params['series'] = res_series

            #save predictor to charts dictionary
            charts[predictor.replace('_', ' ').capitalize()] = params
            
        # Collect States data 
        series = full_df.groupby([y_name, 'state']).count()['age']
        states = dict()
        # We'll calculate max and min values for map coloring
        max_val = 0
        min_val = 100
        for k in series.index.levels[1]:
            if ('yes', k) in series: yes = series[('yes', k)]
            else: yes = 0
            if ('no', k) in series: no = series[('no', k)]
            else: no = 0
            if yes+no > 0:
                val = floor(yes*100/(yes+no))
                states[k] = {"value": val}
                if val > max_val: max_val = val
                if val < min_val: min_val = val
            else: states[k] = {"value": None}        
        params = {'name': 'State distribution'}
        params['states'] = states
        params['minValue'] = min_val
        params['maxValue'] = max_val

        #save predictor to charts dictionary
        charts['State'] = params

        
        
        # Extract target groups from prediction

        # Parse tree and return list of nodes with positive answers (value[id][0][1]>value[id][0][0])
        # Uses global object 'dtc' of type sklearn.tree.DecisionTreeClassifier
        # Return: [[node_id, [path_to_node]]....]
        def travel_tree(node_id, path):
            res = []
            res_l = []
            res_r = []
            new_path_l = path + [(node_id, True)]
            new_path_r = path + [(node_id, False)]
            if dtc.tree_.children_left[node_id] != -1:
                res_l = travel_tree(dtc.tree_.children_left[node_id], new_path_l)
            if dtc.tree_.children_left[node_id] != -1:
                res_r = travel_tree(dtc.tree_.children_right[node_id], new_path_r)

            if dtc.tree_.value[node_id][0][1] > dtc.tree_.value[node_id][0][0]: # if it's positive node
                res = [[node_id, path]]

            return res + res_l + res_r
        # end def travel_tree(node_id, path):
        
        # Desribes path in set of statements
        # Uses global object 'dtc' of type sklearn.tree.DecisionTreeClassifier
        def describe_path(path):
            features = dict()
            for i, truth in path:
                f_name = x2.columns[dtc.tree_.feature[i]]
                if f_name in ['credit_card', 'debt_funds']: # if feature is binary
                    value = 'Has not' if truth else 'Has' # In the tree truth corresponds to value <= 0,5
                    features[f_name] = value
                elif f_name == 'gender':
                    value = 'Female' if truth else 'Male'
                    features[f_name] = value
                elif '=' in f_name: # if feature is category
                    f_name, value = f_name.split('=')
                    if truth: # we do NOT want such value
                        if f_name in features:
                            features[f_name]['not_in'].append(value)
                        else:
                            features[f_name] = {'in': None, 'not_in': [value]}
                    else: #we DO want such value
                        if f_name in features:
                            features[f_name]['in'] = value
                        else:
                            features[f_name] = {'in': value, 'not_in': []}
                else: # feature is numeric
                    value = dtc.tree_.threshold[i]
                    if truth: # it is UPPER limit
                        if f_name in features:
                            features[f_name]['upper'] = value
                        else:
                            features[f_name] = {'lower': None, 'upper': value}
                    else: #it is LOWER limit
                        if f_name in features:
                            features[f_name]['lower'] = value
                        else:
                            features[f_name] = {'lower': value, 'upper': None}
            return features
        #end def describe_path(path):
        
        positives = travel_tree(0, [])
        
        # Sort positives by absolute value, descending
        positives = sorted(positives, key=lambda x: -dtc.tree_.value[x[0]][0][1])
        
        # get some samples (from 10%+1 to 20%+5)
        positives = positives[len(positives)/10+1:len(positives)/5+5]
        
        # sort buy purity, descending
        positives = sorted(positives, 
                key=lambda x: -dtc.tree_.value[x[0]][0][1]/(dtc.tree_.value[x[0]][0][0]+dtc.tree_.value[x[0]][0][1]) )
        # Take no more than first 4
        positives = positives[:4]
        
        targets = []
        for i in range(len(positives)):
            conditions=describe_path(positives[i][1])
            val = dtc.tree_.value[positives[i][0]][0] # will be touple
            percents = floor(val[1]/(val[0]+val[1])*1000)/10
            targets.append({'conditions': conditions, 'percents': percents})
        
        res = jsonify({ 'type': 'model', 'title': '', 'query': query_text, 'features': feature_w, 
                       'charts': charts, 'targets': targets })

    else: # not a model query
        if not query or (not query['sql_select']): 
            return err_msg()

        sql_query = "SELECT " + ",".join(query['sql_select']) + " FROM pci_customers "
        if query['sql_filter']:
            sql_query += " WHERE " + " AND ".join(query['sql_filter'])
        if query['sql_group_by']:
            sql_query += " GROUP BY " + ", ".join(query['sql_group_by']) 
            sql_query += " ORDER BY " + ", ".join(query['sql_group_by'])
        if query['sql_limit']:
            sql_query += " LIMIT " + query['sql_limit']

        chartTitle = "Shown: '" + query['target']
        if query['group_by'] and query['group_by'] != query['target']:
            chartTitle += " by " + query['group_by'] + "'"
        if query['sql_filter']:
            chartTitle += " Filter: " + " AND ".join(query['sql_filter']) + "'"

        if query['group_by'] == "state":
            chartType = "map"
        elif query['group_by'] == "gender":
            chartType = "pie"
            category_ind = query['sql_select'].index('gender')
        elif query['group_by'] == "job":
            chartType = "pie"
            category_ind = query['sql_select'].index('job')
        elif query['group_by'] == "education":
            chartType = "pie"
            category_ind = query['sql_select'].index('education')
        elif query['group_by']:
            chartType = "linear"
        else:
            chartType = "table"
  
        xAxis = []
        yAxis = []
        print sql_query # debug

        conn = psycopg2.connect(database="pci", user="postgres")
        conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
        curs = conn.cursor()

        try:
            curs.execute(sql_query)
        except Exception, e:
            return err_msg()

        rec = curs.fetchall()
        conn.close()

        res = err_msg()

        if chartType == 'pie':
            value_ind = 1 if category_ind == 0 else 0
            for row in rec:
                yAxis.append(round(row[value_ind]))
                xAxis.append(row[category_ind])
            res = jsonify({'type': chartType, 'title': chartTitle, 'xAxis': xAxis, 'yAxis': yAxis, 'query': query_text})

        if chartType == 'linear': 
            for row in rec:
                yAxis.append(round(row[0]))
                xAxis.append(round(row[1]))
            res = jsonify({'type': chartType, 'title': chartTitle, 'xAxis': xAxis, 'yAxis': yAxis,
                          'query': query_text, 'xTitle': query['group_by'], 'yTitle': query['target'] })

        if chartType == 'map':
            states = dict()

            # one of SELECT parameters will be state and other - values
            state_ind = 0 if query['sql_select'][0] == 'state' else 1
            count_ind = 1 if state_ind == 0 else 0

            if len(rec)>0: max_val = rec[0][count_ind] # We'll calculate max and min values for map coloring
            else: max_val = 0
            min_val = max_val
            for row in rec:
                states[row[state_ind]] = {"value": round(row[count_ind])}
                if round(row[count_ind]) > max_val: max_val = round(row[count_ind])
                if round(row[count_ind]) < min_val: min_val = round(row[count_ind])
            res = jsonify({'type': chartType, 'title': chartTitle, 
                           'xAxis': [], 'yAxis': [], 
                           'states': states, 'query': query_text,
                           "maxValue": ceil(max_val), "minValue": floor(min_val)})

    return res;

@app.route('/test_speech')
def test_speechpage():
    try:
        return render_template("test_speech.html")
    except Exception, e:
        return str(e)

@app.route('/')
def welcomepage():
    try:
        return render_template("welcome.html")
    except Exception, e:
        return str(e)


if __name__ == "__main__":
    from gevent.pool import Pool
    from gevent import pywsgi
    from geventwebsocket.handler import WebSocketHandler
    server = pywsgi.WSGIServer(('', 8091), app, handler_class=WebSocketHandler, spawn=Pool())
    server.serve_forever()

1:  [('gender', 18, 24), ('state', 28, 37), ('income', 8, 14)]
5
SELECT
['gender', 'AVG(income)']
WHERE
["state in ('Minnesota')"]
GROUP BY
['gender']
LIMIT

SELECT gender,AVG(income) FROM pci_customers  WHERE state in ('Minnesota') GROUP BY gender ORDER BY gender
1:  [('state', 23, 25), ('debt_funds', 9, 18)]
5
3
SELECT
[]
WHERE
["debt_funds in ('yes')", "state in ('Nevada')"]
GROUP BY
[]
LIMIT



/usr/lib64/python2.7/site-packages/numpy/core/fromnumeric.py:2645: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


1:  [('customer', 24, 29), ('gender', 24, 28), ('state', 33, 41), ('job', 0, 3)]
3
6
SELECT
['job', 'count(*)']
WHERE
["state in ('New York')", "gender in ('M')"]
GROUP BY
['job']
LIMIT

SELECT job,count(*) FROM pci_customers  WHERE state in ('New York') AND gender in ('M') GROUP BY job ORDER BY job
1:  [('gender', 18, 24), ('state', 28, 37), ('income', 8, 14)]
5
SELECT
['gender', 'AVG(income)']
WHERE
["state in ('Minnesota')"]
GROUP BY
['gender']
LIMIT

SELECT gender,AVG(income) FROM pci_customers  WHERE state in ('Minnesota') GROUP BY gender ORDER BY gender
1:  [('gender', 18, 24), ('state', 28, 37), ('income', 8, 14)]
5
SELECT
['gender', 'AVG(income)']
WHERE
["state in ('Minnesota')"]
GROUP BY
['gender']
LIMIT

SELECT gender,AVG(income) FROM pci_customers  WHERE state in ('Minnesota') GROUP BY gender ORDER BY gender
1:  [('customer', 24, 29), ('gender', 24, 28), ('state', 33, 41), ('job', 0, 3)]
3
6
SELECT
['job', 'count(*)']
WHERE
["state in ('New York')", "gender in ('M')"]
GROUP 